In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from ray.tune.schedulers import PopulationBasedTraining
from sklearn.model_selection import KFold
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score

In [2]:
df=pd.read_csv('./cleaned_datasets/bike_data.csv',index_col=0)  

In [3]:
df

,yr,temp,hum,windspeed,cnt,days_since_2011,season_FALL,season_SPRING,season_SUMMER,season_WINTER,holiday_HOLIDAY,workingday_WORKING DAY,weathersit_GOOD,weathersit_MISTY,weathersit_RAIN/SNOW/STORM
0,2011,8.175849,80.5833,10.749882,985,0,0,0,0,1,0,0,0,1,0
1,2011,9.083466,69.6087,16.652113,801,1,0,0,0,1,0,0,0,1,0
2,2011,1.229108,43.7273,16.636703,1349,2,0,0,0,1,0,1,1,0,0
3,2011,1.400000,59.0435,10.739832,1562,3,0,0,0,1,0,1,1,0,0
4,2011,2.666979,43.6957,12.522300,1600,4,0,0,0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2012,3.945849,65.2917,23.458911,2114,726,0,0,0,1,0,1,0,1,0
727,2012,3.906651,59.0000,10.416557,3095,727,0,0,0,1,0,1,0,1,0
728,2012,3.906651,75.2917,8.333661,1341,728,0,0,0,1,0,0,0,1,0
729,2012,4.024151,48.3333,23.500518,1796,729,0,0,0,1,0,0,1,0,0


In [4]:
df=df.drop(columns=['season_SPRING','season_FALL','season_SUMMER','season_WINTER'])

In [5]:
df

,yr,temp,hum,windspeed,cnt,days_since_2011,holiday_HOLIDAY,workingday_WORKING DAY,weathersit_GOOD,weathersit_MISTY,weathersit_RAIN/SNOW/STORM
0,2011,8.175849,80.5833,10.749882,985,0,0,0,0,1,0
1,2011,9.083466,69.6087,16.652113,801,1,0,0,0,1,0
2,2011,1.229108,43.7273,16.636703,1349,2,0,1,1,0,0
3,2011,1.400000,59.0435,10.739832,1562,3,0,1,1,0,0
4,2011,2.666979,43.6957,12.522300,1600,4,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
726,2012,3.945849,65.2917,23.458911,2114,726,0,1,0,1,0
727,2012,3.906651,59.0000,10.416557,3095,727,0,1,0,1,0
728,2012,3.906651,75.2917,8.333661,1341,728,0,0,0,1,0
729,2012,4.024151,48.3333,23.500518,1796,729,0,0,1,0,0


In [6]:
y = df.pop('cnt')
X = df
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)


In [7]:
X_train

,yr,temp,hum,windspeed,days_since_2011,holiday_HOLIDAY,workingday_WORKING DAY,weathersit_GOOD,weathersit_MISTY,weathersit_RAIN/SNOW/STORM
472,2012,20.591651,39.0417,18.333143,472,0,1,1,0,0
330,2011,13.580849,69.8333,13.999918,330,0,0,1,0,0
293,2011,12.092500,57.4167,14.833532,293,0,1,1,0,0
595,2012,23.881651,60.3333,11.917089,595,0,0,1,0,0
728,2012,3.906651,75.2917,8.333661,728,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
16,2011,0.264151,53.7500,12.999139,16,1,0,0,1,0
700,2012,6.021651,80.6667,4.000181,700,0,0,0,1,0
318,2011,16.910000,68.8750,13.375411,318,0,1,0,1,0
676,2012,5.904151,54.7500,20.375236,676,0,1,0,1,0


In [8]:
y_train

472    6691
330    3071
293    4304
595    7865
728    1341
       ... 
16     1000
700    5191
318    4195
676    5035
320    3053
Name: cnt, Length: 584, dtype: int64

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
class DataFrameDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = torch.tensor(X_data, dtype=torch.float32)
        self.y_data = torch.tensor(y_data, dtype=torch.float32)

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [11]:
def create_dataloader(X_train, X_test, y_train, y_test, batch_size=32):
    # Create custom Dataset objects for train and test data
    train_dataset = DataFrameDataset(X_train, y_train)
    test_dataset = DataFrameDataset(X_test, y_test)

    # Create DataLoaders for train and test data
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [12]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, hidden_layers, hidden_units, dropout, dropout_array, input_dim, output_dim, activation=nn.ReLU(),norm=False):
        if hidden_layers != len(hidden_units):
            print("Error: wrong size of hidden_layers or hidden_units")
            return
        layers = []
        i = 0
        if norm:
            layers.append(nn.BatchNorm1d(input_dim))

        super(Net, self).__init__()
        
        layers.append(nn.Linear(input_dim, hidden_units[i]))
        layers.append(activation)
        if dropout:
            layers.append(nn.Dropout(dropout_array[i]))

        for _ in range(hidden_layers - 1):
            i += 1
            layers.append(nn.Linear(hidden_units[i-1], hidden_units[i]))
            layers.append(activation)
            if dropout:
                layers.append(nn.Dropout(dropout_array[i]))

        layers.append(nn.Linear(hidden_units[-1], output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.net(x)


In [13]:
model = Net(hidden_layers=3, hidden_units=[24, 24, 24], dropout=True, dropout_array=[0.2, 0.3, 0.4], input_dim=10, output_dim=1, activation=nn.Sigmoid(),norm=True)


In [14]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
model = model.to(device)

In [15]:
summary(model,input_size=(10,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                   [-1, 10]              20
            Linear-2                   [-1, 24]             264
           Sigmoid-3                   [-1, 24]               0
           Dropout-4                   [-1, 24]               0
            Linear-5                   [-1, 24]             600
           Sigmoid-6                   [-1, 24]               0
           Dropout-7                   [-1, 24]               0
            Linear-8                   [-1, 24]             600
           Sigmoid-9                   [-1, 24]               0
          Dropout-10                   [-1, 24]               0
           Linear-11                    [-1, 1]              25
Total params: 1,509
Trainable params: 1,509
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/back

In [16]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [17]:
def train(config, checkpoint_dir=None):
    hidden_layers=config["hidden_layers"]
    hidden_units=config["hidden_units"]
    dropout=config["dropout"]
    dropout_array=config["dropout_array"]
    activation=config["activation"]
    norm=config["norm"]
    lr=config["lr"]
    input_dim=X_train.shape[1]
    output_dim=1
    
    train_ds = TensorDataset(torch.Tensor(X_train_scaled), torch.Tensor(y_train.to_numpy()))
    test_ds = TensorDataset(torch.Tensor(X_test_scaled), torch.Tensor(y_test.to_numpy()))
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    fold = 0
    avg_loss = 0
    avg_r2 = 0

    for train_idx, val_idx in kfold.split(train_ds):
        print("This is fold :" ,fold)
        #trainloader, testloader = create_dataloader(X_train_scaled, X_test_scaled, y_train.values, y_test.values, batch_size=config["batch_size"])
        net =Net(hidden_layers=hidden_layers, hidden_units=hidden_units, dropout=dropout, dropout_array=dropout_array, input_dim=input_dim, output_dim=output_dim, activation=activation,norm=norm)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        net.to(device)
        criterion = RMSELoss()
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])
        train_subset = torch.utils.data.Subset(train_ds, train_idx)
        val_subset = torch.utils.data.Subset(train_ds, val_idx)

        train_loader = DataLoader(train_subset, batch_size=config["batch_size"], shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=config["batch_size"])
    
        for epoch in range(config["epochs"]):
            running_loss = 0.0
            train_loss=0.0
            val_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            train_loss=running_loss/len(train_loader)
               
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()      
            avg_loss += val_loss / len(val_loader)
            y_true = y_test.to_numpy()
            y_pred = net(torch.Tensor(X_test_scaled).to(device)).cpu()
            r2 = r2_score(y_true, y_pred)
            print(r2)
            tune.report(r2_curr=r2)
            avg_r2 += r2
        fold += 1
        

    avg_loss /= fold
    avg_r2 /= fold
    print("Average loss across folds: ", avg_loss)
    print("Average R2 score across folds: ", avg_r2)
    tune.report(loss=avg_loss,train_loss=train_loss,r2_score=avg_r2,r2_curr=r2)

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [19]:
device

'cuda'

In [20]:
def generate_hidden_units(hidden_layers):
    return np.random.choice(a=[16,32,64,128], size=hidden_layers).tolist()

search_space = {
    "batch_size": tune.choice([8,16,32, 64, 128]),
    "lr": tune.loguniform(1e-7, 1e-2),
    "epochs": tune.choice([25,50, 100, 150,300,500]),
    #"epochs": tune.choice([25]),
    "hidden_layers": tune.randint(1, 4),
    "hidden_units": tune.sample_from(lambda spec: generate_hidden_units(spec.config.hidden_layers)),
    "dropout": tune.choice([True, False]),
    "dropout_array": tune.sample_from(lambda spec: np.random.uniform(low=0.1, high=0.8, size=spec.config.hidden_layers).tolist()),
    "activation": tune.choice([nn.ReLU()]),
    "norm": tune.choice([True, False])
}

In [21]:
scheduler = ASHAScheduler(metric="r2_curr", mode="max", grace_period=2, reduction_factor=2)
"""
pbt_scheduler = PopulationBasedTraining(
    time_attr='training_iteration',
    metric='loss',
    mode='min',
    perturbation_interval=1,
    hyperparam_mutations=search_space
)
"""
reporter = CLIReporter(metric_columns=["loss", "training_iteration","train_loss"])

In [22]:
analysis = tune.run(
        train,
        resources_per_trial={"cpu": 8, "gpu": 1 if torch.cuda.is_available() else 0},
        config=search_space,
        num_samples=100,
        scheduler=scheduler,
        progress_reporter=reporter)

print("Best hyperparameters found were: ", analysis.get_best_config(metric="r2_score", mode="max"))


2023-04-21 12:23:53,527	INFO worker.py:1538 -- Started a local Ray instance.
C:\Users\Patrick\AppData\Roaming\Python\Python39\site-packages\ray\tune\trainable\function_trainable.py:610: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/tune/api_docs/trainable.html

  warnings.warn(


== Status ==
Current time: 2023-04-21 12:23:55 (running for 00:00:00.30)
Memory usage on this node: 11.7/15.9 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.17 GiB heap, 0.0/1.59 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-04-21_12-23-55
Number of trials: 17/100 (16 PENDING, 1 RUNNING)
+-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+
| Trial name        | status   | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |
|-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+------

== Status ==
Current time: 2023-04-21 12:24:09 (running for 00:00:14.78)
Memory usage on this node: 13.6/15.9 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.17 GiB heap, 0.0/1.59 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-04-21_12-23-55
Number of trials: 18/100 (17 PENDING, 1 RUNNING)
+-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+
| Trial name        | status   | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |
|-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+------

Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,r2_curr,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_9e7b2_00000,2023-04-21_12-25-02,True,,12b0133e78f146e8967d6b7047425fe6,DESKTOP-8SILQJU,6,127.0.0.1,12664,-4.97524,62.4979,0.00500369,62.4979,1682072702,0,,6,9e7b2_00000,0.00400329
train_9e7b2_00001,2023-04-21_12-25-20,True,,a5f0fe773d094b17b691bd00a1575b19,DESKTOP-8SILQJU,6,127.0.0.1,13280,0.425241,14.2505,0.004004,14.2505,1682072720,0,,6,9e7b2_00001,0.00400352
train_9e7b2_00002,2023-04-21_12-25-28,True,,c24aacaa5e664fd28464e6a207f8ff77,DESKTOP-8SILQJU,2,127.0.0.1,23540,-5.23812,3.70416,0.980894,3.70416,1682072728,0,,2,9e7b2_00002,0.0038321
train_9e7b2_00003,2023-04-21_12-25-38,True,,9fa77b83359c48eb95b73b64b2551d45,DESKTOP-8SILQJU,2,127.0.0.1,2640,-5.22759,5.70494,2.03085,5.70494,1682072738,0,,2,9e7b2_00003,0.00400352
train_9e7b2_00004,2023-04-21_12-26-08,True,,fcec455af81d446d9a6b4e98ff7c9b35,DESKTOP-8SILQJU,6,127.0.0.1,14868,0.78155,25.489,0.00600529,25.489,1682072768,0,,6,9e7b2_00004,0.00300193
train_9e7b2_00005,2023-04-21_12-26-15,True,,223fd9a23bd54ae9b9cd09e9b96baabe,DESKTOP-8SILQJU,2,127.0.0.1,17508,-5.22982,3.13057,0.681708,3.13057,1682072775,0,,2,9e7b2_00005,0.00400496
train_9e7b2_00006,2023-04-21_12-26-37,True,,fe4155afc1c2482ba58f393b7f2a7ab8,DESKTOP-8SILQJU,4,127.0.0.1,15396,-4.99208,18.5252,4.5183,18.5252,1682072797,0,,4,9e7b2_00006,0.00400448
train_9e7b2_00007,2023-04-21_12-26-44,True,,cefeb04b24e640399503fe2af7e31bc7,DESKTOP-8SILQJU,2,127.0.0.1,19624,-5.23835,2.53331,0.370337,2.53331,1682072804,0,,2,9e7b2_00007,0.00400376
train_9e7b2_00008,2023-04-21_12-26-51,True,,cb13e28c1d084e4991f7da4823efb6de,DESKTOP-8SILQJU,2,127.0.0.1,12924,-5.23777,3.93158,1.16793,3.93158,1682072811,0,,2,9e7b2_00008,0.00400376
train_9e7b2_00009,2023-04-21_12-27-01,True,,93dde98a1acd4992bc0f88ef71e498de,DESKTOP-8SILQJU,2,127.0.0.1,22800,-5.23413,5.42181,1.91826,5.42181,1682072821,0,,2,9e7b2_00009,0.00400281


(train pid=12664) This is fold : 1
== Status ==
Current time: 2023-04-21 12:24:18 (running for 00:00:23.57)
Memory usage on this node: 13.7/15.9 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.17 GiB heap, 0.0/1.59 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-04-21_12-23-55
Number of trials: 18/100 (17 PENDING, 1 RUNNING)
+-------------------+----------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+----------------------+
| Trial name        | status   | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units   |          lr | norm   |   training_iteration |
|-------------------+----------+-----------------+--------------+----

...

2023-04-21 13:19:10,666	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': ReLU()}



(train pid=11944) Average loss across folds:  1095.43359375
(train pid=11944) Average R2 score across folds:  0.7345450952219125
(train pid=22256) This is fold : 0
== Status ==
Current time: 2023-04-21 13:19:15 (running for 00:55:20.69)
Memory usage on this node: 11.5/15.9 GiB 
Using AsyncHyperBand: num_stopped=78
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: -0.40837124012030435 | Iter 2.000: -5.233865346595896
Resources requested: 8.0/16 CPUs, 1.0/1 GPUs, 0.0/3.17 GiB heap, 0.0/1.59 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-04-21_12-23-55
Number of trials: 100/100 (1 RUNNING, 99 TERMINATED)
+-------------------+------------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+----------------+-------------+--------+----------+----------------------+--------------+
| Trial name        | status     | loc             | activation   |   batch_size | d

2023-04-21 13:19:16,925	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'activation': ReLU()}
2023-04-21 13:19:17,088	INFO tune.py:762 -- Total run time: 3322.08 seconds (3321.86 seconds for the tuning loop).


== Status ==
Current time: 2023-04-21 13:19:16 (running for 00:55:21.89)
Memory usage on this node: 11.7/15.9 GiB 
Using AsyncHyperBand: num_stopped=79
Bracket: Iter 64.000: None | Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: -0.40837124012030435 | Iter 2.000: -5.233993905453074
Resources requested: 0/16 CPUs, 0/1 GPUs, 0.0/3.17 GiB heap, 0.0/1.59 GiB objects
Result logdir: C:\Users\Patrick\ray_results\train_2023-04-21_12-23-55
Number of trials: 100/100 (100 TERMINATED)
+-------------------+------------+-----------------+--------------+--------------+-----------+----------------------+----------+-----------------+-----------------+-------------+--------+----------+----------------------+--------------+
| Trial name        | status     | loc             | activation   |   batch_size | dropout   | dropout_array        |   epochs |   hidden_layers | hidden_units    |          lr | norm   |     loss |   training_iteration |   train_loss |
|-------------------+----

Best hyperparameters found were:  {'batch_size': 16, 'lr': 0.003972641114051936, 'epochs': 50, 'hidden_layers': 3, 'hidden_units': [32, 32, 128], 'dropout': False, 'dropout_array': [0.38283509966903684, 0.14655095633529086, 0.5393755103727268], 'activation': ReLU(), 'norm': False}


In [23]:
analysis.results_df.sort_values(by=['r2_score', 'loss'],na_position='first').tail(10)

,loss,train_loss,r2_score,r2_curr,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,...,experiment_tag,config/batch_size,config/lr,config/epochs,config/hidden_layers,config/hidden_units,config/dropout,config/dropout_array,config/activation,config/norm
trial_id,,,,,,,,,,,,,,,,,,,,,
9e7b2_00083,1282.352718,1224.001328,0.662070,0.753484,0.022021,True,None,None,6,3e1a9b2bebdd4bcca7fea9f41851d0ce,...,"83_activation=ReLU,batch_size=8,dropout=False,...",8,0.000025,300,3,"[32, 64, 128]",False,"[0.17570983725368075, 0.20384900363208924, 0.6...",ReLU(),True
9e7b2_00035,1176.315549,1159.879466,0.704421,0.692999,0.012011,True,None,None,6,6a44feff8efb464cb8126df5e0aa18ce,...,"35_activation=ReLU,batch_size=8,dropout=False,...",8,0.000240,150,1,[32],False,[0.16701892223960704],ReLU(),True
9e7b2_00098,1095.433594,1054.986529,0.734545,0.776864,0.026023,True,None,None,6,227ca3dfafb241a19c0569226a259721,...,"98_activation=ReLU,batch_size=16,dropout=False...",16,0.000279,150,2,"[32, 128]",False,"[0.48262717559320933, 0.5388933755647116]",ReLU(),True
9e7b2_00052,1087.981006,989.298407,0.739550,0.740789,0.016015,True,None,None,6,f2448ce469e24d8a8e106e7a77398d91,...,"52_activation=ReLU,batch_size=16,dropout=False...",16,0.000361,150,3,"[16, 64, 128]",False,"[0.27515977640869677, 0.6869359706910142, 0.54...",ReLU(),True
9e7b2_00016,972.441455,940.965141,0.766950,0.767713,0.008016,True,None,None,6,f1f51754b77d479b93db2bfdd68b2418,...,"16_activation=ReLU,batch_size=64,dropout=False...",64,0.001722,100,3,"[32, 64, 16]",False,"[0.12440815535306854, 0.5721148958577107, 0.66...",ReLU(),False
9e7b2_00004,941.255652,877.959030,0.769963,0.781550,0.006005,True,None,None,6,fcec455af81d446d9a6b4e98ff7c9b35,...,"4_activation=ReLU,batch_size=128,dropout=False...",128,0.003741,500,3,"[16, 64, 128]",False,"[0.34681901460022635, 0.5278704824226296, 0.33...",ReLU(),False
9e7b2_00071,985.148816,950.167084,0.772244,0.768699,0.019017,True,None,None,6,77a28edea2024dcca7a3eb8271667ad7,...,"71_activation=ReLU,batch_size=128,dropout=Fals...",128,0.001591,150,3,"[64, 16, 64]",False,"[0.6115229504611344, 0.16563673205008023, 0.59...",ReLU(),True
9e7b2_00078,938.179553,866.360441,0.775567,0.780338,0.020018,True,None,None,6,6b6a8b14182742aeb565039e30379c0e,...,"78_activation=ReLU,batch_size=32,dropout=False...",32,0.001894,300,3,"[16, 32, 128]",False,"[0.2934625770408274, 0.6902749644852175, 0.251...",ReLU(),False
9e7b2_00066,1044.396581,1000.671775,0.782056,0.782194,0.018016,True,None,None,6,623c8c37a40f43b085592248a9e0e428,...,"66_activation=ReLU,batch_size=16,dropout=False...",16,0.002744,150,2,"[16, 64]",False,"[0.46589948716921936, 0.28495098081861264]",ReLU(),True


In [24]:
x=np.linspace(400,10000,101) 
plt.scatter(y_test_pred.detach().numpy(), y_test_con)
plt.plot(x,x,'r')
plt.show()

NameError: name 'y_test_pred' is not defined

In [26]:
cfg = {'batch_size': 16, 'lr': 0.003972641114051936, 'epochs': 50, 'hidden_layers': 3, 'hidden_units': [32, 32, 128], 'dropout': False, 'dropout_array': [0.38283509966903684, 0.14655095633529086, 0.5393755103727268], 'activation': nn.ReLU(), 'norm': False}
hidden_layers=cfg["hidden_layers"]
hidden_units=cfg["hidden_units"]
dropout=cfg["dropout"]
dropout_array=cfg["dropout_array"]
activation=cfg["activation"]
norm=cfg["norm"]
lr=cfg["lr"]

In [27]:
net =Net(hidden_layers=hidden_layers, hidden_units=hidden_units, dropout=dropout, dropout_array=dropout_array, input_dim=14, output_dim=1, activation=activation,norm=norm)

In [28]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
net = net.to(device)

In [29]:
summary(model,input_size=(10,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1                   [-1, 10]              20
            Linear-2                   [-1, 24]             264
           Sigmoid-3                   [-1, 24]               0
           Dropout-4                   [-1, 24]               0
            Linear-5                   [-1, 24]             600
           Sigmoid-6                   [-1, 24]               0
           Dropout-7                   [-1, 24]               0
            Linear-8                   [-1, 24]             600
           Sigmoid-9                   [-1, 24]               0
          Dropout-10                   [-1, 24]               0
           Linear-11                    [-1, 1]              25
Total params: 1,509
Trainable params: 1,509
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/back

In [ ]:
analysis.get_best_checkpoint

In [ ]:
net.load_state_dict(analysis.get_best_checkpoint)

In [33]:
best_config={'batch_size': 16, 'lr': 0.003972641114051936, 'epochs': 50, 'hidden_layers': 3, 'hidden_units': [32, 32, 128], 'dropout': False, 'dropout_array': [0.38283509966903684, 0.14655095633529086, 0.5393755103727268], 'activation': nn.ReLU(), 'norm': False}

In [35]:
best_config

{'batch_size': 16,
 'lr': 0.003972641114051936,
 'epochs': 50,
 'hidden_layers': 3,
 'hidden_units': [32, 32, 128],
 'dropout': False,
 'dropout_array': [0.38283509966903684,
  0.14655095633529086,
  0.5393755103727268],
 'activation': ReLU(),
 'norm': False}

In [34]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def cross_val_train(config, n_splits=5):
    kf = KFold(n_splits=n_splits)
    r2_scores = []
    rmse_scores = []

    for train_index, test_index in kf.split(X_train_scaled, y_train.values):
        X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_test_cv = y_train.values[train_index], y_train.values[test_index]

        train_loader, test_loader = create_dataloader(X_train_cv, X_test_cv, y_train_cv, y_test_cv, batch_size=config["batch_size"])

        input_dim = X_train_cv.shape[1]
        output_dim = 1
        net = Net(hidden_layers=config["hidden_layers"], hidden_units=config["hidden_units"], dropout=config["dropout"], dropout_array=config["dropout_array"], input_dim=input_dim, output_dim=output_dim, activation=config["activation"], norm=config["norm"])

        device = "cuda" if torch.cuda.is_available() else "cpu"
        net.to(device)

        criterion = RMSELoss()
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])

        for epoch in range(config["epochs"]):
            running_loss = 0.0
            for i, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                optimizer.zero_grad()
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            

        with torch.no_grad():
            test_loss = 0.0
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

            y_pred = net(torch.Tensor(X_test_cv).to(device)).cpu().numpy()
            r2_scores.append(r2_score(y_test_cv, y_pred))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test_cv, y_pred)))
            #print(r2_scores)
            #print(rmse_scores)

    avg_r2 = np.mean(r2_scores)
    avg_rmse = np.mean(rmse_scores)

    return avg_r2, avg_rmse
avg_r2, avg_rmse = cross_val_train(best_config)
print(f"Average R2 score: {avg_r2}")
print(f"Average RMSE: {avg_rmse}")


Average R2 score: 0.7468097241548696
Average RMSE: 963.9304611974994


In [ ]:
!pip install deap


In [36]:
best_config

{'batch_size': 16,
 'lr': 0.003972641114051936,
 'epochs': 50,
 'hidden_layers': 3,
 'hidden_units': [32, 32, 128],
 'dropout': False,
 'dropout_array': [0.38283509966903684,
  0.14655095633529086,
  0.5393755103727268],
 'activation': ReLU(),
 'norm': False}

In [38]:
import random
from deap import base, creator, tools, algorithms

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
fixed_config = {
    'hidden_layers': 3,
 'hidden_units': [32, 32, 128],
 'dropout': False,
 'dropout_array': [0.38283509966903684,
  0.14655095633529086,
  0.5393755103727268],
 'activation': nn.ReLU(),
 'norm': False
}

# Define the input variables
toolbox.register("batch_size", random.randint, 8, 100)
toolbox.register("lr", random.uniform, 1e-7, 1e-2)
toolbox.register("epochs", random.randint, 100, 500)

toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.batch_size, toolbox.lr, toolbox.epochs), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_nn(individual):
    config = {'batch_size': int(individual[0]), 'lr': np.abs(individual[1]), 'epochs': int(individual[2])}
    config.update(fixed_config)
    #print(config)

    r2, rmse = cross_val_train(config)
    #print(r2,rmse)
    return rmse

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selBest)
toolbox.register("evaluate", eval_nn)

pop = toolbox.population(n=20)
hof = tools.HallOfFame(1)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, ngen=100, stats=stats, halloffame=hof, verbose=True)

best_individual = hof[0]
print("Best individual: ", best_individual)


C:\Users\Patrick\AppData\Roaming\Python\Python39\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Patrick\AppData\Roaming\Python\Python39\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


KeyboardInterrupt: 

In [ ]:
!pip install scikit-optimize


In [39]:
import numpy as np
from skopt import gp_minimize

fixed_config = {
    'hidden_layers': 3,
 'hidden_units': [32, 32, 128],
 'dropout': False,
 'dropout_array': [0.38283509966903684,
  0.14655095633529086,
  0.5393755103727268],
 'activation': nn.ReLU(),
 'norm': False
}

# The objective function to minimize (negative R^2)
def objective(params):
    batch_size, lr, epochs = params
    config = {'batch_size': int(batch_size), 'lr': lr, 'epochs': int(epochs)}
    config.update(fixed_config)

    r2, rmse = cross_val_train(config)
    return rmse  # Minimize negative R^2 to maximize R^2

# Hyperparameter search space
space = [
    (8, 256),  # batch_size
    (1e-4, 1e-2, 'log-uniform'),  # lr
    (100, 500)  # epochs
]

# Bayesian optimization using Gaussian Process
result = gp_minimize(
    func=objective,
    dimensions=space,
    n_calls=100,  # Number of iterations
    verbose=True
)

# Print the optimized hyperparameters
print("Best hyperparameters found:")
print("Batch size:", result.x[0])
print("Learning rate:", result.x[1])
print("Epochs:", result.x[2])


Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 9.3763
Function value obtained: 954.8615
Current minimum: 954.8615
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 146.6235
Function value obtained: 772.9527
Current minimum: 772.9527
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 6.9904
Function value obtained: 4429.2974
Current minimum: 772.9527
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 4.8254
Function value obtained: 970.7373
Current minimum: 772.9527
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 13.2491
Function value obtained: 1138.0045
Current minimum: 772.9527
Iteration 

Iteration No: 40 ended. Search finished for the next optimal point.
Time taken: 20.8242
Function value obtained: 1039.8533
Current minimum: 772.9527
Iteration No: 41 started. Searching for the next optimal point.
Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 42.8206
Function value obtained: 931.8546
Current minimum: 772.9527
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 21.0618
Function value obtained: 915.7495
Current minimum: 772.9527
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 210.5989
Function value obtained: 776.3377
Current minimum: 772.9527
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 16.5806
Function value obtained: 1122.0229
Current minimum: 772.9527
I

Iteration No: 79 ended. Search finished for the next optimal point.
Time taken: 17.5015
Function value obtained: 843.2667
Current minimum: 765.7200
Iteration No: 80 started. Searching for the next optimal point.
Iteration No: 80 ended. Search finished for the next optimal point.
Time taken: 111.7637
Function value obtained: 779.3139
Current minimum: 765.7200
Iteration No: 81 started. Searching for the next optimal point.
Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 268.9122
Function value obtained: 845.9529
Current minimum: 765.7200
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 112.1228
Function value obtained: 800.5422
Current minimum: 765.7200
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 21.2580
Function value obtained: 951.7591
Current minimum: 765.7200
I